In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
from PIL import Image

# class_id = {
#     "road": 1,
#     "sidewalk": 2,
#     "road roughness": 3,
#     "road boundaries": 4,
#     "crosswalks": 5,
#     "lane": 6,
#     "road color guide": 7,
#     "road marking": 8,
#     "parking": 9,
#     "traffic sign": 10,
#     "traffic light": 11,
#     "pole/structural object": 12,
#     "building": 13,
#     "tunnel": 14,
#     "bridge": 15,
#     "pedestrian": 16,
#     "vehicle": 17,
#     "bicycle": 18,
#     "motorcycle": 19,
#     "personal mobility": 20,
#     "dynamic": 21,
#     "vegetation": 22,
#     "sky": 23,
#     "static": 0
# }

class_id = {
	"motorcycle": 19,
	"personal mobility": 20
}

def get_bounding_boxes_from_png(png_path, target_class_id):
    segmentation_map = np.array(Image.open(png_path))
    bounding_boxes = []
    
    # Find all connected components (contours) for the target class ID
    contours, _ = cv2.findContours((segmentation_map == target_class_id).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        bounding_boxes.append([x, y, x + w, y + h])
    
    return bounding_boxes

def random_crop(image, label, bbox, crop_w=192, crop_h=120, target_w=1920, target_h=1200):
    h, w, _ = image.shape
    x1, y1, x2, y2 = map(int, bbox)
    bbox_w, bbox_h = x2 - x1, y2 - y1
    
    crop_w = max(crop_w, bbox_w)
    crop_h = max(crop_h, bbox_h)
    
    aspect_ratio_w = crop_w
    aspect_ratio_h = crop_h
    
    if aspect_ratio_w / aspect_ratio_h > 8 / 5:
        aspect_ratio_h = int(aspect_ratio_w * 5 / 8)
    else:
        aspect_ratio_w = int(aspect_ratio_h * 8 / 5)
    
    min_crop_x = max(0, x2 - aspect_ratio_w)
    max_crop_x = min(w - aspect_ratio_w, x1)
    min_crop_y = max(0, y2 - aspect_ratio_h)
    max_crop_y = min(h - aspect_ratio_h, y1)
    
    crop_x1 = random.randint(min_crop_x, max_crop_x)
    crop_y1 = random.randint(min_crop_y, max_crop_y)
    crop_x2 = crop_x1 + aspect_ratio_w
    crop_y2 = crop_y1 + aspect_ratio_h
    
    crop_img = image[crop_y1:crop_y2, crop_x1:crop_x2]
    crop_label = label[crop_y1:crop_y2, crop_x1:crop_x2]
    
    # Resize cropped image and label to target size
    resized_crop_img = cv2.resize(crop_img, (target_w, target_h))
    resized_crop_label = cv2.resize(crop_label, (target_w, target_h), interpolation=cv2.INTER_NEAREST)
    
    # # resize한 bbox 좌표 구하기
    # resized_bbox_x1 = int((x1 - crop_x1) * target_w / aspect_ratio_w)
    # resized_bbox_y1 = int((y1 - crop_y1) * target_h / aspect_ratio_h)
    # resized_bbox_x2 = int((x2 - crop_x1) * target_w / aspect_ratio_w)
    # resized_bbox_y2 = int((y2 - crop_y1) * target_h / aspect_ratio_h)
    
    # # resize된 bbox 그리기
    # cv2.rectangle(resized_crop_img, (resized_bbox_x1, resized_bbox_y1), (resized_bbox_x2, resized_bbox_y2), (0, 255, 0), 2)
    
    return resized_crop_img, resized_crop_label

def save_cropped_images_and_labels(file_name, image_file, label_file, target_class, bounding_boxes, image_output_dir, label_output_dir, crop_w=192, crop_h=120):
    image = cv2.imread(image_file)
    label = np.array(Image.open(label_file))
    num_boxes = len(bounding_boxes)
    if num_boxes == 0:
        # print(f"{file_name} : {target_class}에 해당하는 객체가 없습니다.")
        return

    for idx, bbox in enumerate(bounding_boxes):
        cropped_img, cropped_label = random_crop(image, label, bbox, crop_w, crop_h)
        cv2.imwrite(f"{image_output_dir}/cropped{crop_w}{crop_h}_{file_name}_class{class_id[target_class]}_{idx}.jpg", cropped_img)
        Image.fromarray(cropped_label).save(f"{label_output_dir}/cropped{crop_w}{crop_h}_{file_name}_class{class_id[target_class]}_{idx}.png")

## 한 장만으로 돌려보기

In [4]:
# Example usage:
file_name = "S_DRR_230629_008_FC_231"
image_file = f'/mnt/datasets/2DSS/Preprocessed3/images/train/{file_name}.jpg'
png_file = f'/mnt/datasets/2DSS/Preprocessed3/annotations/train/{file_name}.png'

image_output_dir = '/mnt/datasets/2DSS/test/images'
label_output_dir = '/mnt/datasets/2DSS/test/annotations'
target_class = "motorcycle"

crop_size = (960, 600)

bounding_boxes = get_bounding_boxes_from_png(png_file, class_id[target_class])
save_cropped_images_and_labels(file_name, image_file, png_file, target_class, bounding_boxes, image_output_dir, label_output_dir, crop_size[0], crop_size[1])

## 모든 train image에 적용

Motorcycle, Personal Mobility class에 대해 50번 생성

In [6]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
from PIL import Image

# class_id = {
#     "road": 1,
#     "sidewalk": 2,
#     "road roughness": 3,
#     "road boundaries": 4,
#     "crosswalks": 5,
#     "lane": 6,
#     "road color guide": 7,
#     "road marking": 8,
#     "parking": 9,
#     "traffic sign": 10,
#     "traffic light": 11,
#     "pole/structural object": 12,
#     "building": 13,
#     "tunnel": 14,
#     "bridge": 15,
#     "pedestrian": 16,
#     "vehicle": 17,
#     "bicycle": 18,
#     "motorcycle": 19,
#     "personal mobility": 20,
#     "dynamic": 21,
#     "vegetation": 22,
#     "sky": 23,
#     "static": 0
# }

class_id = {
	"motorcycle": 19,
	"personal mobility": 20
}

def get_bounding_boxes_from_png(png_path, target_class_id):
    segmentation_map = np.array(Image.open(png_path))
    bounding_boxes = []
    
    # Find all connected components (contours) for the target class ID
    contours, _ = cv2.findContours((segmentation_map == target_class_id).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        bounding_boxes.append([x, y, x + w, y + h])
    
    return bounding_boxes

def random_crop(image, label, bbox, crop_w=192, crop_h=120, target_w=1920, target_h=1200):
    h, w, _ = image.shape
    x1, y1, x2, y2 = map(int, bbox)
    bbox_w, bbox_h = x2 - x1, y2 - y1
    
    crop_w = max(crop_w, bbox_w)
    crop_h = max(crop_h, bbox_h)
    
    aspect_ratio_w = crop_w
    aspect_ratio_h = crop_h
    
    if aspect_ratio_w / aspect_ratio_h > 8 / 5:
        aspect_ratio_h = int(aspect_ratio_w * 5 / 8)
    else:
        aspect_ratio_w = int(aspect_ratio_h * 8 / 5)
    
    min_crop_x = max(0, x2 - aspect_ratio_w)
    max_crop_x = min(w - aspect_ratio_w, x1)
    min_crop_y = max(0, y2 - aspect_ratio_h)
    max_crop_y = min(h - aspect_ratio_h, y1)
    
    crop_x1 = random.randint(min_crop_x, max_crop_x)
    crop_y1 = random.randint(min_crop_y, max_crop_y)
    crop_x2 = crop_x1 + aspect_ratio_w
    crop_y2 = crop_y1 + aspect_ratio_h
    
    crop_img = image[crop_y1:crop_y2, crop_x1:crop_x2]
    crop_label = label[crop_y1:crop_y2, crop_x1:crop_x2]
    
    # Resize cropped image and label to target size
    resized_crop_img = cv2.resize(crop_img, (target_w, target_h))
    resized_crop_label = cv2.resize(crop_label, (target_w, target_h), interpolation=cv2.INTER_NEAREST)
    
    # # resize한 bbox 좌표 구하기
    # resized_bbox_x1 = int((x1 - crop_x1) * target_w / aspect_ratio_w)
    # resized_bbox_y1 = int((y1 - crop_y1) * target_h / aspect_ratio_h)
    # resized_bbox_x2 = int((x2 - crop_x1) * target_w / aspect_ratio_w)
    # resized_bbox_y2 = int((y2 - crop_y1) * target_h / aspect_ratio_h)
    
    # # resize된 bbox 그리기
    # cv2.rectangle(resized_crop_img, (resized_bbox_x1, resized_bbox_y1), (resized_bbox_x2, resized_bbox_y2), (0, 255, 0), 2)
    
    return resized_crop_img, resized_crop_label

def save_cropped_images_and_labels(iter, file_name, image_file, label_file, target_class, bounding_boxes, image_output_dir, label_output_dir, crop_w=192, crop_h=120):
    image = cv2.imread(image_file)
    label = np.array(Image.open(label_file))
    num_boxes = len(bounding_boxes)
    if num_boxes == 0:
        # print(f"{file_name} : {target_class}에 해당하는 객체가 없습니다.")
        return

    for idx, bbox in enumerate(bounding_boxes):
        cropped_img, cropped_label = random_crop(image, label, bbox, crop_w, crop_h)
        cv2.imwrite(f"{image_output_dir}/cropped{iter}_{crop_w}x{crop_h}_{file_name}_class{class_id[target_class]}_{idx}.jpg", cropped_img)
        Image.fromarray(cropped_label).save(f"{label_output_dir}/cropped{iter}_{crop_w}x{crop_h}_{file_name}_class{class_id[target_class]}_{idx}.png")

In [ ]:
import os

image_dir = '/mnt/datasets/2DSS/Preprocessed3/images/train'
label_dir = '/mnt/datasets/2DSS/Preprocessed3/annotations/train'
image_output_dir = '/mnt/datasets/2DSS/Augmented_19_20/images'
label_output_dir = '/mnt/datasets/2DSS/Augmented_19_20/annotations'

crop_size = (960, 600)

for file_name in os.listdir(image_dir):
	if file_name.endswith('.jpg'):
		file_base_name = file_name.split('.')[0]
		image_file = os.path.join(image_dir, file_name)
		label_file = os.path.join(label_dir, file_base_name + '.png')
			
		if os.path.exists(label_file):
			for class_name, class_idx in class_id.items():
				bounding_boxes = get_bounding_boxes_from_png(label_file, class_idx)
				for i in range(50):
					save_cropped_images_and_labels(i, file_base_name, image_file, label_file, class_name, bounding_boxes, image_output_dir, label_output_dir, crop_size[0], crop_size[1])
		else:
			print(f"Label file does not exist for image: {file_name}")

In [22]:
import os
from tqdm import tqdm

image_dir = '/mnt/datasets/2DSS/Preprocessed3/images/train'
label_dir = '/mnt/datasets/2DSS/Preprocessed3/annotations/train'
image_output_dir = '/mnt/datasets/2DSS/Augmented_19_20/images'
label_output_dir = '/mnt/datasets/2DSS/Augmented_19_20/annotations'

crop_size = (960, 600)

image_files = [file_name for file_name in os.listdir(image_dir) if file_name.endswith('.jpg')]

for file_name in tqdm(image_files, desc="Processing Images", unit="file"):
	file_base_name = file_name.split('.')[0]
	image_file = os.path.join(image_dir, file_name)
	label_file = os.path.join(label_dir, file_base_name + '.png')

	if os.path.exists(label_file):
		for class_name, class_idx in class_id.items():
			bounding_boxes = get_bounding_boxes_from_png(label_file, class_idx)
			if len(bounding_boxes) > 0:
				for i in range(50):
					save_cropped_images_and_labels(
						i, file_base_name, image_file, label_file, class_name, 
						bounding_boxes, image_output_dir, label_output_dir,  
						crop_size[0], crop_size[1]
					)
	else:
		print(f"Label file does not exist for image: {file_name}")

Processing Images: 100%|██████████| 12000/12000 [35:53<00:00,  5.57file/s]  


Vegetation class에 대해 사진 1장 당 1장만 증강되도록

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
from PIL import Image

# class_id = {
#     "road": 1,
#     "sidewalk": 2,
#     "road roughness": 3,
#     "road boundaries": 4,
#     "crosswalks": 5,
#     "lane": 6,
#     "road color guide": 7,
#     "road marking": 8,
#     "parking": 9,
#     "traffic sign": 10,
#     "traffic light": 11,
#     "pole/structural object": 12,
#     "building": 13,
#     "tunnel": 14,
#     "bridge": 15,
#     "pedestrian": 16,
#     "vehicle": 17,
#     "bicycle": 18,
#     "motorcycle": 19,
#     "personal mobility": 20,
#     "dynamic": 21,
#     "vegetation": 22,
#     "sky": 23,
#     "static": 0
# }

class_id = {
	"vegetation":20
}

def get_bounding_boxes_from_png(png_path, target_class_id):
    segmentation_map = np.array(Image.open(png_path))
    bounding_boxes = []
    
    # Find all connected components (contours) for the target class ID
    contours, _ = cv2.findContours((segmentation_map == target_class_id).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        bounding_boxes.append([x, y, x + w, y + h])
    
    return bounding_boxes

def random_crop(image, label, bbox, crop_w=192, crop_h=120, target_w=1920, target_h=1200):
    h, w, _ = image.shape
    x1, y1, x2, y2 = map(int, bbox)
    bbox_w, bbox_h = x2 - x1, y2 - y1
    
    crop_w = max(crop_w, bbox_w)
    crop_h = max(crop_h, bbox_h)
    
    aspect_ratio_w = crop_w
    aspect_ratio_h = crop_h
    
    if aspect_ratio_w / aspect_ratio_h > 8 / 5:
        aspect_ratio_h = int(aspect_ratio_w * 5 / 8)
    else:
        aspect_ratio_w = int(aspect_ratio_h * 8 / 5)
    
    min_crop_x = max(0, x2 - aspect_ratio_w)
    max_crop_x = min(w - aspect_ratio_w, x1)
    min_crop_y = max(0, y2 - aspect_ratio_h)
    max_crop_y = min(h - aspect_ratio_h, y1)
    
    crop_x1 = random.randint(min_crop_x, max_crop_x)
    crop_y1 = random.randint(min_crop_y, max_crop_y)
    crop_x2 = crop_x1 + aspect_ratio_w
    crop_y2 = crop_y1 + aspect_ratio_h
    
    crop_img = image[crop_y1:crop_y2, crop_x1:crop_x2]
    crop_label = label[crop_y1:crop_y2, crop_x1:crop_x2]
    
    # Resize cropped image and label to target size
    resized_crop_img = cv2.resize(crop_img, (target_w, target_h))
    resized_crop_label = cv2.resize(crop_label, (target_w, target_h), interpolation=cv2.INTER_NEAREST)
    
    # # resize한 bbox 좌표 구하기
    # resized_bbox_x1 = int((x1 - crop_x1) * target_w / aspect_ratio_w)
    # resized_bbox_y1 = int((y1 - crop_y1) * target_h / aspect_ratio_h)
    # resized_bbox_x2 = int((x2 - crop_x1) * target_w / aspect_ratio_w)
    # resized_bbox_y2 = int((y2 - crop_y1) * target_h / aspect_ratio_h)
    
    # # resize된 bbox 그리기
    # cv2.rectangle(resized_crop_img, (resized_bbox_x1, resized_bbox_y1), (resized_bbox_x2, resized_bbox_y2), (0, 255, 0), 2)
    
    return resized_crop_img, resized_crop_label

def save_cropped_images_and_labels(iter, file_name, image_file, label_file, target_class, bounding_boxes, image_output_dir, label_output_dir, crop_w=192, crop_h=120):
    image = cv2.imread(image_file)
    label = np.array(Image.open(label_file))
    num_boxes = len(bounding_boxes)
    if num_boxes == 0:
        # print(f"{file_name} : {target_class}에 해당하는 객체가 없습니다.")
        return

    for idx, bbox in enumerate(bounding_boxes):
        cropped_img, cropped_label = random_crop(image, label, bbox, crop_w, crop_h)
        cv2.imwrite(f"{image_output_dir}/cropped{iter}_{crop_w}x{crop_h}_{file_name}_class{class_id[target_class]}_{idx}.jpg", cropped_img)
        Image.fromarray(cropped_label).save(f"{label_output_dir}/cropped{iter}_{crop_w}x{crop_h}_{file_name}_class{class_id[target_class]}_{idx}.png")
        break	# 사진 1장 당 증강 1장

In [5]:
import os
from tqdm import tqdm

image_dir = '/mnt/datasets/2DSS/Preprocessed3/images/train'
label_dir = '/mnt/datasets/2DSS/Preprocessed3/annotations/train'
image_output_dir = '/mnt/datasets/2DSS/Augmented_22/images'
label_output_dir = '/mnt/datasets/2DSS/Augmented_22/annotations'

crop_size = (960, 600)

image_files = [file_name for file_name in os.listdir(image_dir) if file_name.endswith('.jpg')]

for file_name in tqdm(image_files, desc="Processing Images", unit="file"):
	file_base_name = file_name.split('.')[0]
	image_file = os.path.join(image_dir, file_name)
	label_file = os.path.join(label_dir, file_base_name + '.png')

	if os.path.exists(label_file):
		bounding_boxes = get_bounding_boxes_from_png(label_file, 22)
		if len(bounding_boxes) > 0:
			save_cropped_images_and_labels(
				1, file_base_name, image_file, label_file, 'vegetation', 
				bounding_boxes, image_output_dir, label_output_dir,  
				crop_size[0], crop_size[1]
			)
	else:
		print(f"Label file does not exist for image: {file_name}")

Processing Images: 100%|██████████| 12000/12000 [35:02<00:00,  5.71file/s] 
